# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Learning Rate Scheduler
- learning_rate이 너무 높으면 학습이 통통 튀어서 overfitting 일어나
- 너무 낮으면 학습이 어느정도 도달하기 전에 local minimum에 빠져버림
- 그래서 처음엔 높였다가 어느정도 학습이 되면 점점 줄여나가는 방법으로 주로 사용됨

In [14]:
import math

In [15]:
def scheduler(epoch): # epoch을 input으로 넣어
    # 매 epoch마다 sheduler가 learning rate을 바꿔줌
    
    if epoch < 10:
        return 0.001
    
    else: # epoch 10번째 부터 learning rate 점점 줄어듦
        return 0.001 * math.exp(0.01 * (10 - epoch))

In [16]:
for epoch in range(10,20):
    print(0.001 * math.exp(0.01 * (10 - epoch)))
    # learning rate이 점점 작아지는 것을 볼 수 있음

0.001
0.000990049833749168
0.0009801986733067552
0.0009704455335485082
0.0009607894391523232
0.000951229424500714
0.0009417645335842487
0.0009323938199059483
0.0009231163463866358
0.0009139311852712283


In [19]:
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
# verbose=1로 설정해놓으면 learning rate이 줄일 때마다 log를 보여줌

## Training

In [20]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)

# 0.01, 0.001로 기준 learning rate을 살짝 높게 잡은 다음에
# 학습 곡선을 봤을 때 학습이 안되고있을 때 그 epoch을 기억해서
# 그 epoch 기준으로 * 0.1을 해서 0.01이었던 것을 0.001로 하는 등
# 또 학습이 안된다그러면 그 epoch 기억해서 그 기준으로 더 줄이는 등
# 반복학습을 해서 tensorboard를 보다보면 커브를 그려서 보면 언제 learning rate을 줄여야하는지 감이 옴
# 정확히 언제 줄이고 어떻게 줄어야할 지 모르는데 자꾸 공부하다보면 감이 온대..


Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/20
3/3 [==============================] - 3s 1s/step - loss: 2.2504 - accuracy: 0.0588 - val_loss: 2.3329 - val_accuracy: 0.0729

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/20
3/3 [==============================] - 3s 1s/step - loss: 2.3237 - accuracy: 0.0938 - val_loss: 2.3340 - val_accuracy: 0.0729

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Epoch 3/20
3/3 [==============================] - 3s 1s/step - loss: 2.1666 - accuracy: 0.0882 - val_loss: 2.3408 - val_accuracy: 0.0729

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
Epoch 4/20
3/3 [==============================] - 2s 769ms/step - loss: 2.2303 - accuracy: 0.0882 - val_loss: 2.3796 - val_accuracy: 0.0729

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
Epoch 5/20
3/3 [==============================] - 4s 1s/step - loss: 2.3851 - accuracy: 0.0625 - val_los